In [1]:
""" Performing a large-scale experiment on cifar10 """

import keras
import logging
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
import keras.regularizers as regularizers
from keras.models import load_model

# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="swish_first_exp_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def leaky(x):
    return K.maximum(0.3*x, x)

In [3]:
def create(act):
    baseMapNum = 32
    weight_decay = 1e-4

    model = Sequential()
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    return model

In [4]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [5]:
def schedule(x):
    if x < 75: 
        return 0.001
    elif x < 100:
        return 0.0005
    elif x < 125:
        return 0.0003
    elif x<150:
        return 0.00015
    elif x<175:
        return 0.000075
    elif x<200: 
        return 0.000035
    elif x<225:
        return 0.000017
    else:
        return 0.000012

In [11]:
# Create model and get ready - reproducible cell
act, act_name = leaky, "leaky"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('models/leaky_relu/leaky_at_2.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 154s 395ms/step - loss: 0.7549 - acc: 0.7730 - val_loss: 0.6329 - val_acc: 0.8161
Epoch 34/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7490 - acc: 0.7738 - val_loss: 0.6141 - val_acc: 0.8216
Epoch 35/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7473 - acc: 0.7754 - val_loss: 0.5810 - val_acc: 0.8386
Epoch 36/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7389 - acc: 0.7775 - val_loss: 0.5655 - val_acc: 0.8410
Epoch 37/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7412 - acc: 0.7783 - val_loss: 0.5700 - val_acc: 0.8421
Epoch 38/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7299 - acc: 0.7807 - val_loss: 0.5777 - val_acc: 0.8365
Epoch 39/250
390/390 [==============================] - 154s 395ms/step - loss: 0.7374 - acc: 0.7797 - val_loss: 0.5976 - val_acc: 0.8302
Epoch 40/250
390/390 [=========================

390/390 [==============================] - 154s 395ms/step - loss: 0.5747 - acc: 0.8312 - val_loss: 0.4679 - val_acc: 0.8708
Epoch 93/250
390/390 [==============================] - 154s 396ms/step - loss: 0.5703 - acc: 0.8350 - val_loss: 0.4547 - val_acc: 0.8726
Epoch 94/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5770 - acc: 0.8316 - val_loss: 0.4657 - val_acc: 0.8716
Epoch 95/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5697 - acc: 0.8338 - val_loss: 0.4771 - val_acc: 0.8665
Epoch 96/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5691 - acc: 0.8330 - val_loss: 0.4679 - val_acc: 0.8702
Epoch 97/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5677 - acc: 0.8340 - val_loss: 0.4673 - val_acc: 0.8727
Epoch 98/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5696 - acc: 0.8352 - val_loss: 0.4712 - val_acc: 0.8705
Epoch 99/250
390/390 [=========================

390/390 [==============================] - 154s 395ms/step - loss: 0.4586 - acc: 0.8655 - val_loss: 0.3817 - val_acc: 0.8969
Epoch 152/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4505 - acc: 0.8687 - val_loss: 0.3820 - val_acc: 0.8969
Epoch 153/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4513 - acc: 0.8684 - val_loss: 0.3825 - val_acc: 0.8967
Epoch 154/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4518 - acc: 0.8685 - val_loss: 0.3760 - val_acc: 0.8980
Epoch 155/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4490 - acc: 0.8696 - val_loss: 0.3800 - val_acc: 0.8960
Epoch 156/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4472 - acc: 0.8694 - val_loss: 0.3762 - val_acc: 0.8971
Epoch 157/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4478 - acc: 0.8706 - val_loss: 0.3755 - val_acc: 0.8989
Epoch 158/250
390/390 [==================

390/390 [==============================] - 154s 396ms/step - loss: 0.4194 - acc: 0.8785 - val_loss: 0.3655 - val_acc: 0.9033
Epoch 211/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4193 - acc: 0.8779 - val_loss: 0.3629 - val_acc: 0.9019
Epoch 212/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4196 - acc: 0.8778 - val_loss: 0.3645 - val_acc: 0.9022
Epoch 213/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4246 - acc: 0.8755 - val_loss: 0.3642 - val_acc: 0.9027
Epoch 214/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4190 - acc: 0.8764 - val_loss: 0.3657 - val_acc: 0.9009
Epoch 215/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4227 - acc: 0.8781 - val_loss: 0.3676 - val_acc: 0.9025
Epoch 216/250
390/390 [==============================] - 154s 396ms/step - loss: 0.4229 - acc: 0.8768 - val_loss: 0.3646 - val_acc: 0.9020
Epoch 217/250
390/390 [==================

In [6]:
# Create model and get ready - reproducible cell
act, act_name = leaky, "leaky"
model = create(act)
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Training - only for 125 eppochs
batch_size = 128
epochs=250
# lr scheduler
lr_1 = keras.callbacks.LearningRateScheduler(schedule)

# Fit the model
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,verbose=1,validation_data=(x_test,y_test), callbacks=[lr_1])
# Save the model
model.save('models/leaky_relu/leaky_at_3.h5')
# Display logs
print(his.history)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
__________

390/390 [==============================] - 154s 394ms/step - loss: 0.7559 - acc: 0.7708 - val_loss: 0.6157 - val_acc: 0.8208
Epoch 34/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7564 - acc: 0.7719 - val_loss: 0.5777 - val_acc: 0.8366
Epoch 35/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7415 - acc: 0.7770 - val_loss: 0.5707 - val_acc: 0.8409
Epoch 36/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7410 - acc: 0.7778 - val_loss: 0.5970 - val_acc: 0.8287
Epoch 37/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7365 - acc: 0.7777 - val_loss: 0.6265 - val_acc: 0.8236
Epoch 38/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7386 - acc: 0.7800 - val_loss: 0.6287 - val_acc: 0.8206
Epoch 39/250
390/390 [==============================] - 154s 394ms/step - loss: 0.7345 - acc: 0.7794 - val_loss: 0.6056 - val_acc: 0.8251
Epoch 40/250
390/390 [=========================

390/390 [==============================] - 154s 394ms/step - loss: 0.5724 - acc: 0.8347 - val_loss: 0.4670 - val_acc: 0.8711
Epoch 93/250
390/390 [==============================] - 154s 394ms/step - loss: 0.5658 - acc: 0.8353 - val_loss: 0.4676 - val_acc: 0.8720
Epoch 94/250
390/390 [==============================] - 154s 394ms/step - loss: 0.5726 - acc: 0.8330 - val_loss: 0.4584 - val_acc: 0.8747
Epoch 95/250
390/390 [==============================] - 154s 394ms/step - loss: 0.5703 - acc: 0.8352 - val_loss: 0.4690 - val_acc: 0.8732
Epoch 96/250
390/390 [==============================] - 154s 394ms/step - loss: 0.5722 - acc: 0.8324 - val_loss: 0.4423 - val_acc: 0.8789
Epoch 97/250
390/390 [==============================] - 154s 394ms/step - loss: 0.5648 - acc: 0.8350 - val_loss: 0.4614 - val_acc: 0.8756
Epoch 98/250
390/390 [==============================] - 154s 395ms/step - loss: 0.5668 - acc: 0.8348 - val_loss: 0.4466 - val_acc: 0.8803
Epoch 99/250
390/390 [=========================

390/390 [==============================] - 154s 394ms/step - loss: 0.4508 - acc: 0.8674 - val_loss: 0.3747 - val_acc: 0.8983
Epoch 152/250
390/390 [==============================] - 154s 395ms/step - loss: 0.4501 - acc: 0.8684 - val_loss: 0.3777 - val_acc: 0.8972
Epoch 153/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4488 - acc: 0.8699 - val_loss: 0.3742 - val_acc: 0.9002
Epoch 154/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4495 - acc: 0.8696 - val_loss: 0.3709 - val_acc: 0.8970
Epoch 155/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4441 - acc: 0.8702 - val_loss: 0.3732 - val_acc: 0.8999
Epoch 156/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4450 - acc: 0.8695 - val_loss: 0.3719 - val_acc: 0.8985
Epoch 157/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4452 - acc: 0.8690 - val_loss: 0.3732 - val_acc: 0.8981
Epoch 158/250
390/390 [==================

390/390 [==============================] - 154s 394ms/step - loss: 0.4192 - acc: 0.8769 - val_loss: 0.3605 - val_acc: 0.9026
Epoch 211/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4259 - acc: 0.8754 - val_loss: 0.3601 - val_acc: 0.9043
Epoch 212/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4221 - acc: 0.8769 - val_loss: 0.3610 - val_acc: 0.9035
Epoch 213/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4216 - acc: 0.8767 - val_loss: 0.3593 - val_acc: 0.9041
Epoch 214/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4192 - acc: 0.8782 - val_loss: 0.3621 - val_acc: 0.9037
Epoch 215/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4226 - acc: 0.8766 - val_loss: 0.3615 - val_acc: 0.9028
Epoch 216/250
390/390 [==============================] - 154s 394ms/step - loss: 0.4167 - acc: 0.8776 - val_loss: 0.3610 - val_acc: 0.9020
Epoch 217/250
390/390 [==================

In [7]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [12]:
import keras.regularizers as regularizers
ensembler = 0
for i in range(3):
    if i == 0:
        model = create(leaky)
        model.load_weights('models/leaky_relu/leaky_at_'+str(i+1)+'.h5')
        model.save('models/leaky_relu/leaky_at_'+str(i+1)+'.h5')
    else:
        model = load_model('models/leaky_relu/leaky_at_'+str(i+1)+'.h5', custom_objects={"leaky": leaky})
    ensembler += model.predict_proba(x_test)
    
print(accuracy(ensembler, y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_25 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_26 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 32)        0         
__________